## TheBloke/prometheus-13B-v1.0-GPTQ

In [2]:
!pip3 install transformers optimum
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached optimum-1.21.2-py3-none-any.whl.metadata (19 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
Using cached optimum-1.21.2-py3-none-any.whl (424 kB)
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
  Using cached https://huggingface.github.io/autogptq-index/whl/cu118/auto-gptq/auto_gptq-0.7.1%2Bcu118-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (23.7 MB)
  Using cached accelerate-0.32.1-py3-none-any.whl.metadata (18 kB)
  Using cached rouge-1.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached gekko-1.2.1-py3-none-any.whl.meta

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/prometheus-13B-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)



CUDA extension not installed.
CUDA extension not installed.
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/prometheus-13B-v1.0-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.b

In [2]:
import os
import csv

model_name='glm-4v-9b'
with open('scores/'+model_name+'.csv', 'w', newline='') as csvfile:
    fieldnames = ['Image', 'Question', 'Response','Reference_answer', 'Score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    with open('results/'+model_name+'.csv', 'r' , encoding='unicode_escape') as file:
        reader = csv.reader(file)
        next(reader, None)
        for row in reader:
            image = row[0]
            prompt = row[1]
            response = row[2]
            reference_answer = row[3]
            
            prompt_template=f'''###Task Description:
            An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
            1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
            2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
            3. The output format should look as follows: \"Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)\"
            4. Please do not generate any other opening, closing, and explanations.
            
            ###The instruction to evaluate:
            {prompt}
            
            ###Response to evaluate:
            {response}
            
            ###Reference Answer (Score 5):
            {reference_answer}
            
            ###Score Rubrics:
            [An assessment of how well the response addresses the prompt and provides a relevant and accurate. The rubric evaluates the relevance and accuracy in comparison to the reference answer.]
            Score 1: The response is totally irrelevant to the reference answer.
            Score 2: The response has some relevant information but includes details not found in the reference answer.
            Score 3: The response is somewhat similar to the reference answer but lacks depth and detail.
            Score 4: The response is generally similar and accurate but has occasional minor differences.
            Score 5: The response is highly similar, demonstrating a deep understanding and matching the reference answer closely.
            
            ###Feedback:
            '''
            
            # print("\n\n*** Generate:")
            
            input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
            output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
            res= tokenizer.decode(output[0])
            score = res.split("###Feedback")[-1]
            # Inference can also be done using transformers' pipeline
            
            #print("*** Pipeline:")
            # pipe = pipeline(
            #     "text-generation",
            #     model=model,
            #     tokenizer=tokenizer,
            #     max_new_tokens=512,
            #     do_sample=True,
            #     temperature=0.7,
            #     top_p=0.95,
            #     top_k=40,
            #     repetition_penalty=1.1
            # )
            
            # print(pipe(prompt_template)[0]['generated_text'])
            # score = (pipe(prompt_template)[0]['generated_text']).split("###Feedback")[-1]
            writer.writerow({'Image': image, 'Question': prompt, 'Response': response,'Reference_answer': reference_answer,'Score': score })


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad 

In [3]:
import os
import csv

names= ['glm-4v-9b','cogvlm2','internlm-xcomposer2','Mini-InternVL-Chat-2B-V1-5','Mini-InternVL-Chat-4B-V1-5','MiniCPM-Llama3-V-2_5']
for model_name in names:
    with open('scores/'+model_name+'.csv', 'w', newline='') as csvfile:
        fieldnames = ['Image', 'Question', 'Response','Reference_answer', 'Score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        with open('results/'+model_name+'.csv', 'r', encoding='unicode_escape') as file:
            reader = csv.reader(file)
            next(reader, None)
            for row in reader:
                image = row[0]
                prompt = row[1]
                response = row[2]
                reference_answer = row[3]
                
                prompt_template=f'''###Task Description:
                An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
                1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
                2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
                3. The output format should look as follows: \"Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)\"
                4. Please do not generate any other opening, closing, and explanations.
                
                ###The instruction to evaluate:
                {prompt}
                
                ###Response to evaluate:
                {response}
                
                ###Reference Answer (Score 5):
                {reference_answer}
                
                ###Score Rubrics:
                [An assessment of how well the response addresses the prompt and correlates with the reference answer. The rubric evaluates the completeness, relevance, and accuracy of the response in relation to the reference answer.]
                Score 1: The response is significantly inaccurate or irrelevant compared to the reference answer. It includes information that directly contradicts the reference answer or is entirely unrelated to the prompt.
                Score 2: The response contains some relevant information but is mostly incomplete or partially incorrect. It includes several inaccuracies or contradictions in comparison to the reference answer.
                Score 3: The response is somewhat accurate and includes relevant information but lacks completeness. It contains minor errors or omissions and may include some information that is not aligned with the reference answer.
                Score 4: The response is mostly accurate and complete, closely aligning with the reference answer. It may have slight inaccuracies or minor omissions, but it generally provides a correct and relevant answer.
                Score 5: The response is entirely accurate and complete, perfectly aligning with the reference answer. It addresses the prompt thoroughly and without any contradictions or irrelevant information.
                
                ###Feedback:
                '''
                
                # print("\n\n*** Generate:")
                
                input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
                output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
                res= tokenizer.decode(output[0])
                score = res.split("###Feedback")[-1]
                # Inference can also be done using transformers' pipeline
                
                #print("*** Pipeline:")
                # pipe = pipeline(
                #     "text-generation",
                #     model=model,
                #     tokenizer=tokenizer,
                #     max_new_tokens=512,
                #     do_sample=True,
                #     temperature=0.7,
                #     top_p=0.95,
                #     top_k=40,
                #     repetition_penalty=1.1
                # )
                
                # print(pipe(prompt_template)[0]['generated_text'])
                # score = (pipe(prompt_template)[0]['generated_text']).split("###Feedback")[-1]
                writer.writerow({'Image': image, 'Question': prompt, 'Response': response,'Reference_answer': reference_answer,'Score': score })


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad 

### Math

In [6]:

import os
import csv


scores = []
i = 0
with open('scores/moondream2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader, None)
    for row in reader:
        i+=1
        try:
            score = int(row[4].split("[RESULT]")[-1][1])
            scores.append(score)
        except:
            score = row[4]
            print("The feedback is not in the expected format: ",score)
        
print(scores)
print(sum(scores))

The feedback is not in the expected format:  :
            
            The response does accurately describe the steps for changing the engine oil filter of a lawn mower. However, it omits the important step of filling the crankcase with the proper type of new oil, which is crucial to the process. This lack of depth and accuracy compared to the reference answer is why the response is given a score of 3. It is relevant to the prompt, but the omission of this important detail is what keeps it from achieving a higher score. </s>
[3, 4, 4, 1, 2, 2, 1, 2, 5, 2, 1, 2, 3, 2, 5, 1, 2, 1, 1, 1, 2, 4, 2, 1, 2, 2, 2, 1, 2, 1, 2, 3, 4, 3, 3, 3, 2, 3, 2, 3, 3, 3, 1, 1, 2, 3, 5, 1, 1, 3, 1, 3, 1, 3, 1, 1, 1, 1, 4, 3, 1, 1, 2, 5, 3, 3, 4, 2, 3, 1, 3, 3, 3, 4, 4, 5, 3, 2, 4, 3, 2, 1, 4, 3, 2, 2, 1, 2, 3, 1, 2, 2, 5, 1, 1, 1, 3, 5, 2, 1, 3, 1, 4, 2, 3, 3, 3, 1, 2]
259


In [2]:
## The models do not always keep a consistent format
## manually add the scores from the previous cell error message if needed
scores.append(3)

In [2]:
from statistics import mean 
import pandas as pd

df = pd.DataFrame(scores)

print(sum(scores))
print(mean(scores))
print("std", df.std())



323
2.9363636363636365
std 0    0.969983
dtype: float64


In [ ]:
############################

## kaist-ai/prometheus-vision-7b-v1.0

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-to-text", model="kaist-ai/prometheus-vision-13b-v1.0")

In [ ]:
import os
import csv

with open('vision_scores/imp-v1-3b.csv', 'w', newline='') as csvfile:
    fieldnames = ['Image', 'Question', 'Response','Reference_answer', 'Score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    with open('results/imp-v1-3b.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader, None)
        for row in reader:
            image = row[0]
            prompt = row[1]
            response = row[2]
            reference_answer = row[3]
            
            prompt_template=f'''###Task Description:
            An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
            1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
            2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
            3. The output format should look as follows: \"Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)\"
            4. Please do not generate any other opening, closing, and explanations.
            
            ###The instruction to evaluate:
            {prompt}
            
            ###Response to evaluate:
            {response}
            
            ###Reference Answer (Score 5):
            {reference_answer}
            
            ###Score Rubrics:
            [An assessment of how well the response addresses the prompt and provides an accurate and informative explanation. The rubric evaluates the relevance, accuracy, and depth of the response in comparison to the reference answer.]
            Score 1: The response misunderstands the question and provides an irrelevant answer.
            Score 2: The response is relevant but contains misinformation or uses different information from the reference answer.
            Score 3: The response is somewhat relevant but lacks accuracy or depth compared to the reference answer.
            Score 4: The response is mostly accurate and relevant but may lack some detail or clarity compared to the reference answer.
            Score 5: The response provides an accurate and detailed explanation similar to the reference answer, demonstrating a clear understanding of the question.
            
            ###Feedback:
            '''
            
            # print("\n\n*** Generate:")
            
            # input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
            # output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
            # res= tokenizer.decode(output[0])
            # score = res.split("###Feedback")[-1]
            # Inference can also be done using transformers' pipeline
            
            #print("*** Pipeline:")
            # pipe = pipeline(
            #     "text-generation",
            #     model=model,
            #     tokenizer=tokenizer,
            #     max_new_tokens=512,
            #     do_sample=True,
            #     temperature=0.7,
            #     top_p=0.95,
            #     top_k=40,
            #     repetition_penalty=1.1
            # )
            
            # print(pipe(prompt_template)[0]['generated_text'])
            score = (pipe(prompt_template)[0]['generated_text']).split("###Feedback")[-1]
            writer.writerow({'Image': image, 'Question': prompt, 'Response': response,'Reference_answer': reference_answer,'Score': score })


## TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GPTQ

In [1]:
!pip3 install --upgrade transformers optimum
# If using PyTorch 2.1 + CUDA 12.x:
!pip3 install --upgrade auto-gptq
# or, if using PyTorch 2.1 + CUDA 11.x:
# !pip3 install --upgrade auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/


Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


CUDA extension not installed.
CUDA extension not installed.
/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.

In [3]:
import os
import csv

model_name ='glm-4v-9b'
with open( 'dolp_scores/'+model_name+'.csv', 'w', newline='') as csvfile:
    fieldnames = ['Image', 'Question', 'Response','Reference_answer', 'Score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with open('results/'+ model_name+'.csv', 'r' , encoding='unicode_escape') as file:
        reader = csv.reader(file)
        next(reader, None)
        for row in reader:
            image = row[0]
            question = row[1]
            response = row[2]
            reference_answer = row[3]
            
            prompt = f'''###Task Description:
            An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
            1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
            2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
            3. The output format should look as follows: \"Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)\"
            4. Please do not generate any other opening, closing, and explanations.

            ###Score Rubrics:
            [Is the model proficient in applying relevant and semantic similarity responses to the reference answer?"]
            "score1_description":"The response is totally irrelevant to the reference answer.",
            "score2_description":"The response has some relevant information but includes details not found in the reference answer.",
            "score3_description":"The response is somewhat similar to the reference answer but lacks depth and detail.",
            "score4_description":"The response is generally similar and accurate but has occasional minor differences.",
            "score5_description":"The response is highly similar, demonstrating a deep understanding and matching the reference answer closely."

            
            ###The question: "How do I set the timing in my watch according to the image?".
            ###Response to evaluate: "1. Pull out the crown.
            2. Turn it to adjust the time. 3. Push the crown back in to secure the setting."
            ###Reference Answer (Score 5): "To set the timing on your watch, follow these general steps: 1. Pull the crown (the small knob on the side of the watch) out.
            2. Turn the crown clockwise or counterclockwise to adjust the hands to the correct time.
            3. Once the time is set, push the crown back into its original position to secure the setting."
            ### Feedback: The response is related to the question and reference answer, giving similar information to the reference answer (not misleading information),
            and implies accuracy and a deep understanding of the question. Therefore, the Score (digit from 1 to 5) is ###Score: 5<end>
            
            ###The question: "What is the name of part 2 in the chair image?"
            ###Response to evaluate: "2"
            ###Reference Answer (Score 5): "Arm pads"
            ### Feedback: The response doesn't even name a part of a chair. The response is not similar to the reference answer and is not related. Therefore, the Score (digit from 1 to 5) is ###Score: 1<end>

            ###The question:
            {question}
            
            ###Response to evaluate:
            {response}
            
            ###Reference Answer (Score 5):
            {reference_answer}
            
            ### Feedback:'''
            
            system_message = "You are a judge evaluating the response depending on the reference answer"
            prompt_template=f'''<|im_start|>system
            {system_message}<|im_end|>
            <|im_start|>user
            {prompt}<|im_end|>
            <|im_start|>assistant
            '''
            
            input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
            output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
            score = tokenizer.decode(output[0])
            writer.writerow({'Image': image, 'Question': question, 'Response': response,'Reference_answer': reference_answer,'Score': score })


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad 

In [8]:
import os
import csv

result= 0
i = 0
with open('dolp_scores/moondream2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader, None)
    for row in reader:
        
        try:
            score = int(((row[4].split("assistant")[-1]).strip())[0])
            result += score
        except:
            score = (row[4].split("assistant")[-1])
            print( i,score)
        i+=1
        
            

print(result)

3 
            
            Feedback: The response is mostly accurate and relevant but may lack some detail or clarity compared to the reference answer. It provides a good starting point by identifying the two instructions in the image, but the actual instructions for disabling the auto restart feature are only partially correct. The correct instructions should be: "1. Open the front cover. 2. Lift up both sides of the cover slightly. 3. Press the On/Off button and hold it for 6 seconds. The unit will beep twice and the lamp will blink 6 times."

[4]<|im_end|>
4 
            
Feedback: The response accurately addresses the prompt and provides an accurate and informative explanation similar to the reference answer, demonstrating a clear understanding of the question. It includes the necessary steps for cleaning the air conditioner's air filter, such as turning off the power, unplugging it, and removing the filter for cleaning. The response also emphasizes the importance of properly reas

In [17]:

import os
import csv


scores = []
i = 0
with open('dolp_scores/moondream2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader, None)
    for row in reader:
        i+=1
        try:
            score = int(((row[4].split("assistant")[-1]).strip())[0])
            scores.append(score)
        except:
            score = (row[4].split("assistant")[-1])
            print("The feedback is not in the expected format: ",score)
        
print(scores)

[4, 5, 5, 4, 5, 5, 4, 4, 4, 5, 4, 4, 4, 5, 5, 5, 4, 3, 4, 4, 4, 5, 4, 5, 5, 5, 5, 2, 5, 4, 4, 5, 5, 4, 4, 4, 5, 4, 5, 4, 5, 5, 5, 5, 4, 4, 4, 5, 5, 4, 4, 5, 5, 5, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 5, 4, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 5, 4, 5, 4, 5, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 5, 5, 5, 4, 4, 4, 4, 5, 4, 4, 5, 5]
483


In [12]:
## The models do not always keep a consistent format
## manually add the scores from the previous cell error message if needed
scores.append(5)

In [13]:
from statistics import mean 
import pandas as pd

df = pd.DataFrame(scores)

print(sum(scores))
print(mean(scores))
print("std", df.std())

407
3.803738317757009
std 0    0.636014
dtype: float64


## prometheus-eval/prometheus-7b-v2.0
https://huggingface.co/prometheus-eval/prometheus-7b-v2.0

In [1]:
!pip install prometheus-eval

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached prometheus_eval-0.1.18-py3-none-any.whl.metadata (26 kB)
  Using cached aiolimiter-1.1.0-py3-none-any.whl.metadata (4.5 kB)
  Using cached fastchat-0.1.0-py3-none-any.whl.metadata (195 bytes)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached openai-1.37.0-py3-none-any.whl.metadata (22 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using ca

In [2]:
!pip install vllm

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached vllm-0.5.3.post1-cp38-abi3-manylinux1_x86_64.whl.metadata (1.8 kB)
  Using cached cmake-3.30.1-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.1 kB)
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
  Using cached fastapi-0.111.1-py3-none-any.whl.metadata (26 kB)
  Using cached uvicorn-0.30.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached prometheus_fastapi_instrumentator-7.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached lm_format_enforcer-0.10.3-py3-none-any.whl.metadata (16 kB)
  Using cached outlines-0.0.46-py3-none-any.whl.metadata (15 kB)
  Using cached ray-2.32.0-cp310-cp310-manylinux2014_x86_6

In [3]:
# Absolute Grading: Outputs score of 1 to 5

from prometheus_eval.vllm import VLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE

model = VLLM(model="prometheus-eval/prometheus-7b-v2.0")
judge = PrometheusEval(model=model, absolute_grade_template=ABSOLUTE_PROMPT)


INFO 07-24 18:48:40 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='prometheus-eval/prometheus-7b-v2.0', speculative_config=None, tokenizer='prometheus-eval/prometheus-7b-v2.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=prometheus-eval/prometheus-7b-v2.0, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 07-24 18:48:41 model_runner.py:680] Starting to load model prometheus-eval/prometheus-7b-v2.0...

Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


INFO 07-24 18:48:59 model_runner.py:692] Loading model weights took 13.4966 GB
INFO 07-24 18:49:02 gpu_executor.py:102] # GPU blocks: 9134, # CPU blocks: 2048
INFO 07-24 18:49:03 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-24 18:49:03 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-24 18:49:09 model_runner.py:1181] Graph capturing finished in 6 secs.


In [4]:
import os
import csv


model_name ='glm-4v-9b'

rubric_data = {
  "criteria":"Is the model proficient in applying relevant and semantic similarity responses to the reference answer?",
  "score1_description":"The response is totally irrelevant to the reference answer.",
  "score2_description":"The response has some relevant information but includes details not found in the reference answer.",
  "score3_description":"The response is somewhat similar to the reference answer but lacks depth and detail.",
  "score4_description":"The response is generally similar and accurate but has occasional minor differences.",
  "score5_description":"The response is highly similar, demonstrating a deep understanding and matching the reference answer closely."
}

score_rubric = SCORE_RUBRIC_TEMPLATE.format(**rubric_data)

with open( 'bad_prompt/'+model_name+'.csv', 'w', newline='') as csvfile:
    fieldnames = ['Image', 'Question', 'Response','Reference_answer','Feedback','Score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    with open('results/'+ model_name+'.csv', 'r' , encoding='unicode_escape') as file:
        reader = csv.reader(file)
        next(reader, None)
        for row in reader:
            image = row[0]
            question = row[1]
            # response = row[2]
            # reference_answer = row[3]

            instruction = "## Reference_answer:"+ row[3],
            response = row[2] ,
            reference_answer = row[3],
            feedback, score = judge.single_absolute_grade(
                instruction=instruction,
                response=response,
                rubric=score_rubric,
                reference_answer=reference_answer
            )
            print(score)
            writer.writerow({'Image': image, 'Question': question, 'Response': response,'Reference_answer': reference_answer,'Feedback':feedback,'Score': score })


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.73s/it, est. speed input: 273.33 toks/s, output: 69.61 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 9845.78it/s]


1


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.73s/it, est. speed input: 267.31 toks/s, output: 70.77 toks/s]


...

1


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.44s/it, est. speed input: 393.68 toks/s, output: 69.98 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 11428.62it/s]

1


In [5]:
## all models
import os
import csv

rubric_data = {
  "criteria":"Is the model proficient in applying relevant and semantic similarity responses to the reference answer?",
  "score1_description":"The response is totally irrelevant to the reference answer.",
  "score2_description":"The response has some relevant information but includes details not found in the reference answer.",
  "score3_description":"The response is somewhat similar to the reference answer but lacks depth and detail.",
  "score4_description":"The response is generally similar and accurate but has occasional minor differences.",
  "score5_description":"The response is highly similar, demonstrating a deep understanding and matching the reference answer closely."
}

score_rubric = SCORE_RUBRIC_TEMPLATE.format(**rubric_data)

names= ['cogvlm2','internlm-xcomposer2','Mini-InternVL-Chat-2B-V1-5','Mini-InternVL-Chat-4B-V1-5','MiniCPM-Llama3-V-2_5']
for model_name in names:
    
    with open( 'prom7b_scores/'+model_name+'.csv', 'w', newline='') as csvfile:
        fieldnames = ['Image', 'Question', 'Response','Reference_answer','Feedback','Score']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
        with open('results/'+ model_name+'.csv', 'r' , encoding='unicode_escape') as file:
            reader = csv.reader(file)
            next(reader, None)
            for row in reader:
                image = row[0]
                question = row[1]
                # response = row[2]
                # reference_answer = row[3]
    
                instruction = "## Reference_answer: "+ row[3],
                response = row[2] ,
                reference_answer = row[3],
                feedback, score = judge.single_absolute_grade(
                    instruction=instruction,
                    response=response,
                    rubric=score_rubric,
                    reference_answer=reference_answer
                )
                print(score)
                writer.writerow({'Image': image, 'Question': question, 'Response': response,'Reference_answer': reference_answer,'Feedback':feedback,'Score': score })


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.91s/it, est. speed input: 169.79 toks/s, output: 71.29 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 5190.97it/s]


2


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.52s/it, est. speed input: 223.78 toks/s, output: 70.62 toks/s]


...

4


Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.04s/it, est. speed input: 184.74 toks/s, output: 71.33 toks/s]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 9058.97it/s]

2


In [ ]:
## relative 
# Relative Grading: Outputs A or B

from prometheus_eval.vllm import VLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import RELATIVE_PROMPT

model = VLLM(model="prometheus-eval/prometheus-7b-v2.0")
judge = PrometheusEval(model=model, relative_grade_template=RELATIVE_PROMPT)


data = {
  "instruction": "A group of historians are conducting a debate on the factors that led to the fall of the Roman Empire. One historian argues that the primary reason for the fall was the constant pressure from barbarian invasions. Another one believes it was because of economic troubles and overreliance on slave labor. A third one suggests it was due to moral decay and political instability. Each historian needs to provide evidence to support their claims. How would the historian arguing for economic troubles and overreliance on slave labor present their case?",
  "response_A": "The historian arguing that economic troubles and overreliance on slave labor led to the fall of the Roman Empire would say this: The Empire's economy was heavily affected by the devaluation of Roman currency. This currency debasement resulted in rampant inflation, disrupting the stability of the economy. Additionally, the Roman Empire heavily depended on slave labor. This caused unemployment among free citizens because maintaining slaves was cheaper than hiring free citizens. The decline in employment opportunities resulted in economic instability. On top of these, the empire's expansion towards the east made them reliant on imports, like grain from Egypt. This over-dependency on imports caused a trade deficit, which further weakened the economy. As the empire lost territories, maintaining the trade imbalance became difficult, causing economic downfall. Thus, the economic troubles and overreliance on slave labor were among the main reasons for the fall of the Roman Empire.",
  "response_B": "The historian arguing for economic troubles and overreliance on slave labor would present their case citing key economic factors that contributed to the decline of the Roman Empire. Harper (2016) outlined how the devaluation of Roman currency led to inflation, disrupting economic stability. Additionally, Scheidel (2007) emphasized that the overuse of slaves resulted in widespread unemployment among free citizens, destabilizing the economy further. The empire's dependency on grain imports from Egypt, creating a trade deficit as highlighted by Temin (2006), also contributed to the economic decline. Thus, the combination of these factors played a crucial role in the fall of the Roman Empire.",
  "reference_answer": "This argument focuses on the economic troubles and overreliance on slave labor as primary reasons for the fall of the Roman Empire. To start with, one of the significant pieces of evidence is the devaluation of Roman currency. As highlighted by Harper (2016), the empire suffered from severe inflation due to the constant debasement of their currency, making it difficult for the economy to remain stable. Moreover, the overreliance on slave labor also played a detrimental role. As pointed out by Scheidel (2007), the dependence on slaves led to unemployment among free Roman citizens. This is because slaves were significantly cheaper to maintain compared to hiring free citizens, leading to a decline in job opportunities, which in turn resulted in economic instability. Furthermore, the empire's expansion to the east made them highly dependent on imports, for instance, grain from Egypt. As noted by Temin (2006), this created a trade deficit that further weakened the Roman economy. When the empire began to lose its territories, it became increasingly difficult to maintain this trade imbalance, leading to economic decline. In conclusion, it can be argued that the economic troubles, mainly due to the devaluation of currency and overreliance on slave labor, were significant contributing factors to the fall of the Roman Empire. The evidence provided, which includes scholarly references to Harper (2016), Scheidel (2007), and Temin (2006), supports this thesis.",
  "rubric": "Is the answer well supported with evidence, including citations/attributions wherever relevant?"
}


feedback, score = judge.single_relative_grade(**data)

print("Feedback:", feedback)
print("Score:", score)

# Output
# Feedback: Both Response A and Response B correctly identify economic troubles and overreliance on slave labor as significant contributing factors to the fall of the Roman Empire. However, Response B is more effective in presenting the historian's argument due to its inclusion of scholarly sources to back up its claims. Specifically, it references works by Harper, Scheidel, and Temin, which adds credibility to the historian's argument and aligns well with the score rubric's emphasis on evidence and citations. While Response A provides a similar argument, it lacks any form of citations or attributions, which lessens the strength of the evidence presented. Therefore, based on the provided rubric, Response B is the superior response due to its use of scholarly evidence to support the historian's claims.
# Score: B